# Get Learning Cruves

The goal of this jupyter notebook is to get the learning curves accuracies in tran, validation an test for the baseline model and MLNT-teacher and MLNT-student neural networks for each epoch.

It is intended to be used in google colab.
A folder named *learning_curves* needs to be created in order to store the results.
It retrieves data from the checkpoint folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/pfm/')

# Pytorch libraries
import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# Internal files
import config
import dataloader
import models
from baseline import get_model, save_checkpoint

import math
import os
import sys
import time
from collections import OrderedDict
import random

import pandas as pd


noise file noisy_label_kv.txt generated with noise: 0.5



In [3]:
# Get the original_dataset
loader = dataloader.KeyDataLoader()
train_loader, valid_loader, test_loader = loader.run()

# Hyper Parameter settings
random.seed(config.seed)
# torch.cuda.set_device(config.gpuid)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
use_cuda = torch.cuda.is_available()

# Networks setup
print('\nModel setup')
print('| Building network: {}'.format(config.model))
net = get_model()

if use_cuda:
  net.cuda()

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:704: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")



Model setup
| Building network: resnet50


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
def load_checkpoint(epoch):
  checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + "_" + str(epoch) + '.pth.tar')
  net.load_state_dict(checkpoint['state_dict'])

In [0]:
def test(loader):
    net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = net(inputs).cuda()

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Grab validation results
    test_acc = 100. * correct/total

    return(round(float(test_acc),2))

In [0]:
def get_accy_list(max_epoch, loader):
  max_epoch = 101

  accy_list = []
  for epoch in range(1,101):
    load_checkpoint(epoch)
    accy_list.append(test(loader))
  return(accy_list)
    # print(accy_list)

In [10]:
print("computing test list\n")
test_list  = get_accy_list(config.num_epochs, test_loader)
print(test_list)
print("computing val list\n")
valid_list = get_accy_list(config.num_epochs, valid_loader)
print(valid_list)
print("computing train list\n")
train_list = get_accy_list(config.num_epochs, train_loader)

df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

computing test list

[83.9, 87.72, 88.38, 88.81, 89.26, 89.27, 89.9, 89.84, 88.98, 89.55, 90.62, 90.41, 90.23, 90.18, 90.98, 90.07, 90.94, 90.23, 90.9, 89.6, 90.41, 89.75, 89.93, 89.72, 89.59, 88.99, 89.42, 89.27, 90.14, 89.24, 89.08, 88.91, 88.02, 88.16, 88.3, 87.41, 87.21, 85.99, 85.86, 84.82, 83.5, 85.77, 85.05, 82.17, 84.05, 82.13, 81.89, 79.9, 77.64, 78.25, 76.33, 76.47, 74.07, 77.46, 73.41, 72.24, 73.27, 70.65, 71.35, 69.69, 67.08, 63.45, 66.11, 65.76, 65.2, 63.04, 62.1, 62.35, 59.76, 56.8, 59.03, 60.04, 60.38, 60.81, 59.46, 54.74, 54.15, 56.55, 59.02, 55.04, 55.12, 54.47, 55.98, 54.39, 55.23, 54.88, 54.22, 56.1, 51.92, 54.11, 54.19, 53.46, 52.1, 55.63, 54.72, 51.98, 53.26, 53.03, 58.39, 51.74]
computing val list

[84.18, 88.38, 89.42, 89.72, 90.16, 90.08, 90.24, 90.28, 89.28, 90.12, 91.12, 90.62, 90.98, 90.7, 91.3, 90.58, 91.48, 90.68, 91.44, 90.18, 91.22, 90.28, 90.36, 90.42, 90.04, 89.74, 90.46, 89.66, 90.68, 89.74, 89.9, 89.5, 88.68, 88.86, 88.98, 87.76, 87.94, 86.06, 86.48, 

In [13]:
df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

df

,epoch,test,valid,train
0,1,83.90,84.18,37.92
1,2,87.72,88.38,40.47
2,3,88.38,89.42,40.80
3,4,88.81,89.72,40.99
4,5,89.26,90.16,41.23
...,...,...,...,...
95,96,51.98,53.88,45.78
96,97,53.26,54.78,45.88
97,98,53.03,54.12,45.91
98,99,58.39,59.58,46.48


In [0]:
clean_kv = config.drive_dir +"/learning_curves/"+ config.checkpoint+".csv"
# clean_kv = "/content/drive/My Drive/Colab_Notebooks/pfm/data/asdf.txt"
df.to_csv(clean_kv, sep=';', index=False,header = True)